From https://towardsdatascience.com/multi-label-image-classification-in-tensorflow-2-0-7d4cf8a4bc72

In [158]:
import tensorflow as tf
import json
import pandas as pd
from pathlib import Path
import os

In [159]:
data_info = pd.read_csv(os.path.join("D:", "collection", "artwork_data.csv"))
data_info

C:\Users\Ludovica\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,accession_number,artist,artistRole,artistId,title,dateText,medium,creditLine,year,acquisitionYear,dimensions,width,height,depth,units,inscription,thumbnailCopyright,thumbnailUrl,url
0,1035,A00001,"Blake, Robert",artist,38,A Figure Bowing before a Seated Old Man with h...,date not known,"Watercolour, ink, chalk and graphite on paper....",Presented by Mrs John Richmond 1922,NaN,1922.0,support: 394 x 419 mm,394,419,NaN,mm,NaN,NaN,http://www.tate.org.uk/art/images/work/A/A00/A...,http://www.tate.org.uk/art/artworks/blake-a-fi...
1,1036,A00002,"Blake, Robert",artist,38,"Two Drawings of Frightened Figures, Probably f...",date not known,Graphite on paper,Presented by Mrs John Richmond 1922,NaN,1922.0,support: 311 x 213 mm,311,213,NaN,mm,NaN,NaN,http://www.tate.org.uk/art/images/work/A/A00/A...,http://www.tate.org.uk/art/artworks/blake-two-...
2,1037,A00003,"Blake, Robert",artist,38,The Preaching of Warning. Verso: An Old Man En...,?c.1785,Graphite on paper. Verso: graphite on paper,Presented by Mrs John Richmond 1922,1785,1922.0,support: 343 x 467 mm,343,467,NaN,mm,NaN,NaN,http://www.tate.org.uk/art/images/work/A/A00/A...,http://www.tate.org.uk/art/artworks/blake-the-...
3,1038,A00004,"Blake, Robert",artist,38,Six Drawings of Figures with Outstretched Arms,date not known,Graphite on paper,Presented by Mrs John Richmond 1922,NaN,1922.0,support: 318 x 394 mm,318,394,NaN,mm,NaN,NaN,http://www.tate.org.uk/art/images/work/A/A00/A...,http://www.tate.org.uk/art/artworks/blake-six-...
4,1039,A00005,"Blake, William",artist,39,The Circle of the Lustful: Francesca da Rimini...,"1826–7, reprinted 1892",Line engraving on paper,Purchased with the assistance of a special gra...,1826,1919.0,image: 243 x 335 mm,243,335,NaN,mm,NaN,NaN,http://www.tate.org.uk/art/images/work/A/A00/A...,http://www.tate.org.uk/art/artworks/blake-the-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69196,122960,T13865,"P-Orridge, Genesis",artist,16646,Larvae (from Tampax Romana),1975,"Perspex, Wood, hairpiece, tampon and human blood",Transferred from Tate Archive 2012,1975,2013.0,object: 305 x 305 x 135 mm,305,305,135.0,mm,NaN,NaN,NaN,http://www.tate.org.uk/art/artworks/p-orridge-...
69197,122961,T13866,"P-Orridge, Genesis",artist,16646,Living Womb (from Tampax Romana),1976,"Wood, Perspex, plastic, photograph on paper, t...",Transferred from Tate Archive 2012,1976,2013.0,object: 305 x 305 x 135 mm,305,305,135.0,mm,NaN,NaN,NaN,http://www.tate.org.uk/art/artworks/p-orridge-...
69198,121181,T13867,"Hatoum, Mona",artist,2365,Present Tense,1996,Soap and glass beads,Presented by Tate Members 2013,1996,2013.0,displayed: 45 x 2410 x 2990 mm,45,2410,2990.0,mm,NaN,NaN,NaN,http://www.tate.org.uk/art/artworks/hatoum-pre...
69199,112306,T13868,"Creed, Martin",artist,2760,Work No. 227: The lights going on and off,2000,Gallery lighting,"Purchased with funds provided by Tate Members,...",2000,2013.0,Overall display dimensions variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.tate.org.uk/art/artworks/creed-work...


In [160]:
with open('TateDictLevel1.json', 'r') as infile:
    tree1 = json.load(infile)
len(tree1.values())

16

In [161]:
len(tree1['people'])

15469

In [162]:
'''
Get a list of keys from dictionary which has the given value
'''
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = list()
    for item in dictOfElements.items():
        listOfItems.append(item)
    for item in listOfItems:
        if valueToFind in item[1]:
            listOfKeys.append((valueToFind, item[0]))
    
    return listOfKeys

In [163]:
values = data_info.accession_number.tolist()
tuples = list()

for i in range(len(values)):
    tuples.append(getKeysByValue(tree1, values[i]))
    
    
print(len(tuples))

69201


In [164]:
class2num = {'people':0, 'objects':1, 'places':2, 'architecture':3, 'abstraction':4, 'society':5, 'nature':6, \
             'emotions, concepts and ideas':7, 'interiors':8, 'work and occupations':9, 'symbols & personifications':10, \
             'religion and belief':11, 'leisure and pastimes':12, 'history':13, 'literature and fiction':14, 'group/movement':15}
class2num['architecture']

3

In [167]:
import numpy
img_target = []

for tupl in tuples:
    if len(tupl) > 0:
        zarray = numpy.zeros(16)
        for i in range(len(tupl)):
            zarray[class2num[tupl[i][1]]] = 1
        img_target.append((tupl[0][0], zarray))
        
len(img_target)

26968

In [168]:
IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

In [169]:
def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    print(image_decoded)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    # Decode it into a dense vector
    return image_normalized, label

In [170]:
BATCH_SIZE = 256 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations

In [171]:
import numpy as np
def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    filenames = np.asarray(filenames)
    labels = np.asarray(labels)
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    print(dataset.element_spec)
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [172]:
mypath = os.path.join('..','..','data_tate')
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [198]:
filenames = []
labels = []
for i,img_targ in enumerate(img_target):
    img_targ0 = str(img_targ[0])+'_8.jpg'
    if img_targ0 in onlyfiles:
        filenames.append(str(img_targ[0])+'_8.jpg')
        labels.append(img_targ[1])

In [199]:
filenames = [os.path.join('..','..','data_tate',str(filename)) \
                 for filename in filenames]
print(len(filenames))

24999


In [200]:
import pickle
with open('filenames.pkl', 'wb') as outfile:
    pickle.dump(filenames, outfile)
    
with open('labels.pkl', 'wb') as outfile2:
    pickle.dump(labels, outfile2)

In [201]:
with open('filenames.pkl', 'rb') as infile:
    filenames = pickle.load(infile)
    
with open('labels.pkl', 'rb') as infile2:
    labels = pickle.load(infile2)
    
print(len(filenames), len(labels))

24999 24999


In [202]:
train_ds = create_dataset(filenames, labels)
train_ds

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(16,), dtype=tf.float64, name=None))
Tensor("DecodeJpeg:0", shape=(None, None, 3), dtype=uint8)


<PrefetchDataset shapes: ((None, 224, 224, 3), (None, 16)), types: (tf.float32, tf.float64)>

In [203]:
import tensorflow_hub as hub

feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [196]:
feature_extractor_layer.trainable = False

In [129]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(1024, activation='relu', name='hidden_layer'),
    tf.keras.layers.Dense(16, activation='sigmoid', name='output')
])

In [130]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [131]:
LR = 1e-5 # Keep it small when transfer learning
EPOCHS = 30

In [132]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=tf.keras.losses.binary_crossentropy,
  metrics=[macro_f1]
)

In [204]:
history = model.fit(train_ds,
  epochs=EPOCHS,)

Epoch 1/30
98/98 [==============================] - 1065s 11s/step - loss: 0.5584 - macro_f1: 0.1676
Epoch 2/30
98/98 [==============================] - 1008s 10s/step - loss: 0.4540 - macro_f1: 0.1414
Epoch 3/30
98/98 [==============================] - 997s 10s/step - loss: 0.4270 - macro_f1: 0.1722
Epoch 4/30
98/98 [==============================] - 994s 10s/step - loss: 0.4088 - macro_f1: 0.2008
Epoch 5/30
98/98 [==============================] - 1008s 10s/step - loss: 0.3949 - macro_f1: 0.2204
Epoch 6/30
98/98 [==============================] - 1005s 10s/step - loss: 0.3845 - macro_f1: 0.2371
Epoch 7/30
79/98 [=======================>......] - ETA: 3:23:30 - loss: 0.3822 - macro_f1: 0.2484

KeyboardInterrupt: 

In [ ]:
tf.io.read_file(os.path.join('..','..','data_tate','A00001_8.jpg'))